## Assignments

To close out this checkpoint, you're going to do three assignments. For the first assignment, you'll write up a short answer to a question.  For the second two assignments, you'll do your work in Jupyter notebooks.


Please submit links to all your work below. This is not a graded checkpoint, but you should discuss your solutions with your mentor. Also, when you're done, compare your work to [these example solutions](https://github.com/Thinkful-Ed/machine-learning-regression-problems/blob/master/notebooks/4.solution_understanding_the_relationship.ipynb).

### 1. Interpretation and significance

Suppose that we would like to know how much families in the US are spending on recreation annually. We've estimated the following model:

$$ expenditure = 873 + 0.0012annual\_income + 0.00002annual\_income^2 - 223.57have\_kids $$

*expenditure* is the annual spending on recreation in US dollars, *annual_income* is the annual income in US dollars, and *have_kids* is a dummy variable indicating the families with children. Interpret the estimated coefficients. What additional statistics should be given in order to make sure that your interpretations make sense statistically. Write up your answer.

The constant in this model is 873 with an annual income coefficient of 0.0012 and annual income 2 coefficient of 0.00002 and a have kids coefficient of -223.57. This model is fitted at 873 where it crosses the Y-axis. Additionally it appears that families with kids spend 223.57 less than those without on recreation not considering annual income. We would want to run t-tests and p-values on each coefficient to see their statistical significance to interpret value derived from them in the model.

Additionally there are 2 annual income features that appears to add additional spend on recreation for each extra dollar earned in annual income. 

### 2. Weather model

In this exercise, you'll work with the historical temperature data from the previous checkpoint. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

* First, load the dataset from the **weatherinszeged** table from Thinkful's database.
* Build a linear regression model where your target variable is the difference between the *apparenttemperature* and the *temperature*. As explanatory variables, use *humidity* and *windspeed*. Now, estimate your model using OLS. Are the estimated coefficients statistically significant? Are the signs of the estimated coefficients in line with your previous expectations? Interpret the estimated coefficients. What are the relations between the target and the explanatory variables? 
* Next, include the interaction of *humidity* and *windspeed* to the model above and estimate the model using OLS. Are the coefficients statistically significant? Did the signs of the estimated coefficients for *humidity* and *windspeed* change? Interpret the estimated coefficients.

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model
import statsmodels.formula.api as smf
from sqlalchemy import create_engine
import statsmodels.api as sm

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

import warnings
warnings.filterwarnings(action="ignore")

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

weather = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

weather.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472,7.389,0.890,14.120,251.000,15.826,0.000,1015.130,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.356,7.228,0.860,14.265,259.000,15.826,0.000,1015.630,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.378,9.378,0.890,3.928,204.000,14.957,0.000,1015.940,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.289,5.944,0.830,14.104,269.000,15.826,0.000,1016.410,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.756,6.978,0.830,11.045,259.000,15.826,0.000,1016.510,Partly cloudy throughout the day.


In [8]:
# Y is the target variable
Y = weather['apparenttemperature'] - weather['temperature']

# X is the feature set 
X = weather[['humidity','windspeed']]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Fri, 03 Jan 2020   Prob (F-statistic):               0.00
Time:                        07:56:31   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The bias in this model is 2.4381

The coeffecient for humidity is -3.0292 which means that for each 10% increase in humidity the difference between the temperature and apparent temperature decreases 0.3029 degrees.

The coeffecient for windspeed is -0.1193 which means that for each mph increase in windspeed the difference between the temperature and apparent temperature dereases 0.1193 degrees.

Both variables are statistically significant according to their p-values. However the coeffecients for humidity and windspeed are negative which is not what I would expect.

### Interaction of Humidity and Windspeed

In [9]:
# This is the interaction between bmi and smoking
weather["humidity_windspeed"] = weather.humidity * weather.windspeed

# X is the feature set
X = weather[['humidity','windspeed', 'humidity_windspeed']]

# We add a constant to the model as it's a best practice
# to do so every time!
X = sm.add_constant(X)

# We fit an OLS model using statsmodels
results = sm.OLS(Y, X).fit()

# We print the summary results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Fri, 03 Jan 2020   Prob (F-statistic):               0.00
Time:                        08:00:14   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0839      0

We can see above that our bias has now changed to 0.0839, coeffecient for humidity has changed to positive 0.1775, coffecient for windspeed as changed to positive 0.0905, and our new interaction feature of humidity_windspeed has a coeffecient of -0.2971. Essentially the combination of windspeed and humitidty have the effect of canceling each other out.

A 1 point increase in humidity changes the target by 0.1775 and a 1 point increase in windspeed increases the target by 0.0905

All of our features are still statistically significant in this model. 

With the inclusion of the interaction feature we can interpret it as follows. Given a windspeed level, 1 point increase in humidity results in 0.18 - 0.30 X windspeed point increase in the target. This means that the increase in the target is lower for high values of windspeed than for low values of windspeed. So, the windspeed mitigates the effect of humidity increase on the target. Similarly for a given humidity level, 1 point increase in the windspeed results in 0.09 - 0.30 X humidity point increase in the target. So, the humidity also mitigates the effect of windspeed on the target.

###  3. House prices model

In this exercise, you'll interpret your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

* Load the **houseprices** data from Thinkful's database.
* Run your house prices model again and interpret the results. Which features are statistically significant, and which are not?
* Now, exclude the insignificant features from your model. Did anything change?
* Interpret the statistically significant coefficients by quantifying their relations with the house prices. Which features have a more prominent effect on house prices?
* Do the results sound reasonable to you? If not, try to explain the potential reasons.

In [16]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import seaborn as sns
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
houseprices = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


houseprices.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.000,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.000,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.000,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.000,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.000,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [17]:
# Drop fields with greater than 10% missing data
houseprices = houseprices.drop(['poolqc', 'miscfeature', 'alley', 'fence','fireplacequ', 'lotfrontage'], axis=1)
#use inplace=true instead of replacing dataframe - cleaner way

#impute missing data with most frequent value (mode) for remaining missing data
houseprices = houseprices.apply(lambda x:x.fillna(x.value_counts().index[0]))

# create a dataframe that holds only categorical features
cat_houseprices = houseprices.select_dtypes(include=['object']).copy()

# Use label encoder to convert categorical variables into numerical
cat_houseprices_label = cat_houseprices.apply(LabelEncoder().fit_transform)

#combine label encoded categorical data with saleprice to get correlation matrix
cat_houseprices_label['saleprice'] = houseprices['saleprice']

# Build out dataframe with selected features

houseprices = pd.concat([houseprices, pd.get_dummies(houseprices.foundation, prefix="foundation", drop_first=True)], axis=1)
houseprices = pd.concat([houseprices, pd.get_dummies(houseprices.centralair, prefix="centralair", drop_first=True)], axis=1)
dummy_col = list(pd.get_dummies(houseprices.foundation, prefix="foundation", drop_first=True).columns)
dummy_col = dummy_col + list(pd.get_dummies(houseprices.centralair, prefix="centralair", drop_first=True).columns)


In [18]:

X = houseprices[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf', 'fullbath'] + dummy_col]
Y = houseprices.saleprice
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.766
Method:                 Least Squares   F-statistic:                     398.8
Date:                Fri, 03 Jan 2020   Prob (F-statistic):               0.00
Time:                        08:11:36   Log-Likelihood:                -17478.
No. Observations:                1460   AIC:                         3.498e+04
Df Residuals:                    1447   BIC:                         3.505e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const             -1.078e+05   6343.394    -16.995      0.000    -1.2e+05   -9.54e+04
overallqual        2.187e+04   1183.004     18.487      0.000    1.95e+04    2.42e+04
grlivarea            48.2604      2.835     17.026      0.000      42.700      53.821
garagecars         1.232e+04   3055.293      4.031      0.000    6323.488    1.83e+04
garagearea           16.2194     10.430      1.555      0.120      -4.240      36.679
totalbsmtsf          31.6675      3.047     10.394      0.000      25.691      37.644
fullbath          -2072.6936   2604.845     -0.796      0.426   -7182.371    3036.984
foundation_CBlock  1.145e+04   3736.147      3.063      0.002    4116.568    1.88e+04
foundation_PConc   2.037e+04   4063.437      5.012      0.000    1.24e+04    2.83e+04
foundation_Slab    3.019e+04   8903.980      3.391      0.001    1.27e+04    4.77e+04
foundation_Stone    803.0377   1.61e+04      0.050      0.960   -3.07e+04    3.23e+04
foundation_Wood   -8784.9596   2.25e+04     -0.391      0.696   -5.29e+04    3.53e+04
centralair_Y       8597.4867   4506.508      1.908      0.057    -242.500    1.74e+04
==============================================================================
Omnibus:                      465.157   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            50678.009
Skew:                          -0.403   Prob(JB):                         0.00
Kurtosis:                      31.852   Cond. No.                     4.48e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.48e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Many of the variables selected are not statistically significant, including garagearea, fullbath, foundation_stone, foundation_wood, and central_air

- for each point increase in overallqual the saleprice increases by 2,187
- for each point increase in grlivarea the saleprice increases by 48.26
- for each point increase in garagecars the saleprice increases by 1,232
- for each point increase in totalbsmtsf the saleprice increases by 31.67

Exclude non-statistically significant variables as well as categorical variables that I botched and re-run model

In [22]:
# fit new model
X = houseprices[['overallqual', 'grlivarea', 'garagecars', 'totalbsmtsf']]
Y = houseprices.saleprice
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.761
Model:                            OLS   Adj. R-squared:                  0.760
Method:                 Least Squares   F-statistic:                     1156.
Date:                Fri, 03 Jan 2020   Prob (F-statistic):               0.00
Time:                        08:21:41   Log-Likelihood:                -17500.
No. Observations:                1460   AIC:                         3.501e+04
Df Residuals:                    1455   BIC:                         3.504e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -9.925e+04   4639.866    -21.390      0.000   -1.08e+05   -9.01e+04
overallqual  2.357e+04   1072.465     21.980      0.000    2.15e+04    2.57e+04
grlivarea      45.6432      2.484     18.376      0.000      40.771      50.515
garagecars   1.858e+04   1747.412     10.634      0.000    1.52e+04     2.2e+04
totalbsmtsf    32.5203      2.838     11.459      0.000      26.953      38.087
==============================================================================
Omnibus:                      421.692   Durbin-Watson:                   1.969
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            36798.858
Skew:                          -0.273   Prob(JB):                         0.00
Kurtosis:                      27.589   Cond. No.                     8.95e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.95e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Coeffecients stayed relatively constant with the changes to the model with the largest change to garagecars